In [ ]:
import pandas as pd
import covid19_module

In [ ]:
df_cases_raw = pd.read_csv("./COVID-19_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", index_col=False)
df_deaths_raw = pd.read_csv("./COVID-19_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", index_col=False)
df_recoveries_raw = pd.read_csv("./COVID-19_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv", index_col=False)
df_population_raw = pd.read_csv("./population_data/API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv")
#df_cases_raw.head()

In [ ]:
df_cases = covid19_module.preprocess_frame(df_cases_raw)
df_deaths = covid19_module.preprocess_frame(df_deaths_raw)
df_recoveries = covid19_module.preprocess_frame(df_recoveries_raw)
df_population = df_population_raw[["Country Name", "2018"]]
#df_cases.head()

In [ ]:
top_cases = covid19_module.top_countries(df_cases_raw)
top_deaths = covid19_module.top_countries(df_deaths_raw)

covid19_module.print_mortality_rates(df_cases, df_deaths, top_deaths)
covid19_module.print_percentage_infected(df_cases, df_population, top_cases)
covid19_module.print_permil_deaths(df_deaths, df_population, top_deaths)

In [ ]:
covid19_module.shift_to_day_zero(df_cases, df_cases)
covid19_module.shift_to_day_zero(df_deaths, df_deaths)
covid19_module.shift_to_day_zero(df_recoveries, df_recoveries)

In [ ]:
%matplotlib notebook

countries = top_deaths
covid19_module.plot_confirmed_cases(df_deaths, countries)

In [ ]:
%matplotlib notebook

covid19_module.plot_case_death_recovery('China', df_cases, df_deaths, df_recoveries)
covid19_module.plot_case_death_recovery('notChina', df_cases, df_deaths, df_recoveries)

In [ ]:
%matplotlib notebook

covid19_module.fit_cases_data("Italy", df_cases)

In [ ]:
%matplotlib notebook

covid19_module.fit_cases_data_sigmoid("Italy", df_cases)

In [ ]:
%matplotlib notebook
interval = 7
covid19_module.plot_daily_vs_total(df_cases, "China", interval)
covid19_module.plot_daily_vs_total(df_cases, "Korea, South", interval)
covid19_module.plot_daily_vs_total(df_cases, "Greece", interval)
covid19_module.plot_daily_vs_total(df_cases, "US", interval)
covid19_module.plot_daily_vs_total(df_cases, "Italy", interval)
covid19_module.plot_daily_vs_total(df_cases, "Spain", interval)
covid19_module.plot_daily_vs_total(df_cases, "Sweden", interval)